In [1]:
import numpy as np
import os
os.getcwd()

data = np.load("../data/processed/supervised_train.npz")
X, y = data["X"], data["y"]

print("X shape:", X.shape)          # Expect (N, 12, 8, 8)
print("y shape:", y.shape)          # Expect (N,)
print("dtype:", X.dtype, y.dtype)
print("action_space:", y.max() + 1, "max index:", y.max())

X shape: (348113, 12, 8, 8)
y shape: (348113,)
dtype: float32 int64
action_space: 4661 max index: 4660


In [3]:
import sys
from pathlib import Path
import numpy as np
import torch
import os
os.chdir("C:/Users/salva/.jupyter/cs372-chess-bot")
print("Changed cwd to:", os.getcwd())
# THEN add src to path
repo_root = Path("C:/Users/salva/.jupyter/cs372-chess-bot").resolve()
sys.path.insert(0, str(repo_root / "src"))

from chess_engine.action_encoding import ACTION_SIZE, encode_move_index
from chess_engine.state_encoding import board_to_tensor
from chess_engine.ChessGame import ChessGame
from chess_nnet.NNetWrapper import NNetWrapper
from core.MCTS import MCTS
from envs.chess_gym import GymChessEnv

def basic_dataset_checks(npz_path="data/processed/supervised_train.npz"):
    data = np.load("data/processed/supervised_train.npz")
    X, y = data["X"], data["y"]
    print("X.shape", X.shape, "dtype", X.dtype)
    print("y.shape", y.shape, "dtype", y.dtype)
    print("min/max y:", int(y.min()), int(y.max()))
    assert X.ndim == 4 and X.shape[1:] == (12,8,8)
    assert y.ndim == 1 and len(y) == X.shape[0]
    assert np.isfinite(X).all(), "NaN/Inf in X"
    assert np.isfinite(y).all(), "NaN/Inf in y"
    assert (y >= 0).all() and (y < ACTION_SIZE).all(), "labels out of range"
    print("unique label count:", np.unique(y).size, "/", ACTION_SIZE)

def model_and_mcts_smoke():
    game = ChessGame()
    nnet = NNetWrapper(game)
    sample_board = game.getInitBoard()
    canon = game.getCanonicalForm(sample_board, 1)
    board_tensor = board_to_tensor(canon).astype(np.float32)
    pi, v = nnet.predict(board_tensor)
    print("predict -> pi.shape", np.shape(pi), "v", v)
    assert np.shape(pi)[0] == ACTION_SIZE
    mcts = MCTS(game, nnet, type("Args", (), {"numMCTSSims":10, "cpuct":1.0})())
    probs = mcts.getActionProb(canon, temp=1)
    print("mcts probs shape", np.shape(probs), "sum", np.sum(probs))
    assert np.isclose(np.sum(probs), 1, atol=1e-6)

def gym_smoke():
    env = GymChessEnv()
    obs, info = env.reset()
    print("env.reset obs shape", np.array(obs).shape)
    board = env._board
    player = env._player
    valid = env.game.getValidMoves(board, player)
    act = int(np.argmax(valid)) if valid.sum()>0 else 0
    obs2, reward, term, trunc, info = env.step(act)
    print("env.step -> reward", reward, "term", term, "trunc", trunc)


print(basic_dataset_checks())
print(model_and_mcts_smoke())
print(gym_smoke())

Changed cwd to: C:\Users\salva\.jupyter\cs372-chess-bot
X.shape (348113, 12, 8, 8) dtype float32
y.shape (348113,) dtype int64
min/max y: 0 4660
unique label count: 1807 / 4672


All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.


None
predict -> pi.shape (4672,) v [0.02305662]


All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.


mcts probs shape (4672,) sum 1.0
None
env.reset obs shape (12, 8, 8)
env.step -> reward 0.0 term False trunc False
None


In [1]:
import sys
from pathlib import Path
import numpy as np
import traceback

repo_root = Path("C:/Users/salva\.jupyter\cs372-chess-bot").resolve()
sys.path.insert(0, str(repo_root / "src"))

import subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "python-chess", "-q"])

import chess
from chess_engine.action_encoding import ACTION_SIZE
from chess_engine.state_encoding import board_to_tensor
from chess_engine.ChessGame import ChessGame
from chess_nnet.NNetWrapper import NNetWrapper
from core.MCTS import MCTS

print("=== EXTENDED DIAGNOSTIC ===\n")

try:
    print("1-4. [skipping, we know these work]\n")
    
    game = ChessGame()
    board = game.getInitBoard()
    canon = game.getCanonicalForm(board, 1)
    tensor = board_to_tensor(canon)
    
    print("5. Creating NNetWrapper...")
    nnet = NNetWrapper(game)
    print("   ✓ Success\n")
    
    print("6. Calling nnet.predict(tensor)...")
    pi, v = nnet.predict(tensor)
    print(f"   ✓ Success")
    print(f"   pi shape: {np.shape(pi)}, v: {v}\n")
    
    print("7. Creating MCTS...")
    args = type("Args", (), {"numMCTSSims": 10, "cpuct": 1.0})()
    mcts = MCTS(game, nnet, args)
    print("   ✓ Success\n")
    
    print("8. Calling mcts.getActionProb(canon, temp=1)...")
    probs = mcts.getActionProb(canon, temp=1)
    print(f"   ✓ Success")
    print(f"   probs shape: {np.shape(probs)}, sum: {np.sum(probs)}\n")
    
except Exception as e:
    print(f"\n✗ ERROR at step above:")
    print(f"  {type(e).__name__}: {e}\n")
    traceback.print_exc()

=== EXTENDED DIAGNOSTIC ===

1-4. [skipping, we know these work]

5. Creating NNetWrapper...


All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.


   ✓ Success

6. Calling nnet.predict(tensor)...
   ✓ Success
   pi shape: (4672,), v: [-0.02676481]

7. Creating MCTS...
   ✓ Success

8. Calling mcts.getActionProb(canon, temp=1)...


All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.
All valid moves were masked, doing a workaround.


   ✓ Success
   probs shape: (4672,), sum: 1.0

